In [ ]:
# ======================================
# 01_download_and_prepare.ipynb
# Descarga datos crudos, construye features y etiquetas,
# y guarda datasets procesados por ticker.
# ======================================

from src.utils import load_config, ensure_dirs
from src.data_loader import load_universe_data
from src.features import build_features
from src.labeler import make_labels

import os
import pandas as pd
import pprint

# --- 1. cargar config y asegurar estructura ---
cfg = load_config()
ensure_dirs()

print("=== CONFIG ESTRATEGIA ===")
pprint.pprint(cfg["strategy"])

# --- 2. leer tickers ---
with open("data/tickers.txt") as f:
    tickers = [line.strip() for line in f if line.strip()]

print("=== TICKERS ===")
print(tickers)

# --- 3. descargar datos crudos 1m ---
raw_map = load_universe_data(
    tickers,
    cfg["data"]["lookback_days"],
    cfg["data"]["interval"]
)

# --- 4. generar features y etiquetas por ticker ---
os.makedirs("data/processed", exist_ok=True)

processed_info = []

for t, df in raw_map.items():
    print(f"\nProcesando {t}...")

    # features técnicos / contexto intradía
    feat_df = build_features(df)

    # etiquetas basadas en TP / SL / horizon
    labeled_df = make_labels(
        feat_df,
        cfg["strategy"]["horizon"],
        cfg["strategy"]["tp"],
        cfg["strategy"]["sl"]
    )

    # guardar dataset etiquetado para inspección offline
    out_path = f"data/processed/{t}_processed.csv"
    labeled_df.to_csv(out_path, index=True)
    print(f"✅ Guardado {out_path} ({len(labeled_df)} filas)")

    processed_info.append({
        "ticker": t,
        "rows": len(labeled_df),
        "path": out_path
    })

print("\n=== RESUMEN DATASETS GENERADOS ===")
for info in processed_info:
    print(info)

print("\nListo. Ahora puedes revisar los CSV en data/processed/ o pasar al notebook 02_train_models.ipynb.")
